In [463]:
import pandas as pd
import numpy as np
import os
import pygsheets
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

In [464]:
os.getcwd()

'D:\\demandes_ponctuelles_py\\RD'

In [465]:
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = 'client_secrets.json'
VIEW_ID = '121230559'
SHEET_ID = ''

In [466]:
# For the full list of dimensions & metrics, check https://developers.google.com/analytics/devguides/reporting/core/dimsmets
DIMENSIONS = ['ga:dimension4']
METRICS = ['ga:avgSessionDuration','ga:pageviewsPerSession','ga:screenviewsPerSession']
#METRICS = ['ga:sessions','ga:avgSessionDuration','ga:pageviews','ga:avgTimeOnPage','ga:screenviews','ga:avgScreenviewDuration']

In [467]:
pageToken = 'go'
store_response = {}

In [468]:
def initialize_analyticsreporting():
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics


In [469]:
def get_report(analytics):
    return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'pageToken': pageToken,
          'pageSize': 100000,
          'dateRanges': [{'startDate': '2018-01-01', 'endDate': '2018-12-31'}],
          'metrics': [{'expression':i} for i in METRICS],
          'dimensions': [{'name':j} for j in DIMENSIONS],
          "dimensionFilterClauses": [
        {
          "filters": [
            {
              "dimensionName": "ga:dimension2",
              "operator": "REGEXP",
              "expressions": ["abonne"]
            }
          ]
        }
        ],
          'includeEmptyRows': 'true',
          "samplingLevel":  "LARGE"
        }],
          "useResourceQuotas": "true"
      }
  ).execute()


In [470]:
while pageToken != None:
    
        # debugging, and it's working with me :p
        print(pageToken)
        analytics = initialize_analyticsreporting()
        response = get_report(analytics)

        store_response[pageToken] = response

        #pageToken = response['reports'][0]['nextPageToken'] # update the pageToken
        pageToken = response['reports'][0].get('nextPageToken')
        #query['reportRequests'][0]['pageToken'] = pageToken # not needed here cause we fetch the query directly


go


In [471]:
def convert_to_dataframe(store_response):
    # init empty df
    dataFrameFormat = pd.DataFrame()  
    
    # one more loop cause each data got back from pageToken is a dictionary inside a bigger one named 'go','50000','100000'...
    for i in list(store_response.keys()):
        for report in store_response.get(i,{}).get('reports', []):
            columnHeader = report.get('columnHeader', {})
            dimensionHeaders = columnHeader.get('dimensions', [])
            metricHeaders = [j.get('name',{}) for j in columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])]
            finalRows = []
    

            for row in report.get('data', {}).get('rows', []):
                dimensions = row.get('dimensions', [])
                metrics = row.get('metrics', [])[0].get('values', {})
                rowObject = {}

                for header, dimension in zip(dimensionHeaders, dimensions):
                    rowObject[header] = dimension
        
                for metricHeader, metric in zip(metricHeaders, metrics):
                    rowObject[metricHeader] = metric

                finalRows.append(rowObject)
      
            dataFrameFormat = pd.concat([dataFrameFormat, pd.DataFrame(finalRows)])
    return dataFrameFormat

In [472]:
def export_to_sheets(df):
    gc = pygsheets.authorize(service_file='client_secrets.json')
    sht = gc.open_by_key(SHEET_ID)
    wks = sht.worksheet_by_title('Sheet1')
    wks.set_dataframe(df,'A1')


In [473]:
#def main():
#analytics = initialize_analyticsreporting()
#response = get_report(analytics)
df = convert_to_dataframe(store_response)   #df = pandas dataframe
 # export_to_sheets(df)            ./      #outputs to spreadsheet. comment to skip
  #print(df)                             

#if __name__ == '__main__':
 # main()

In [474]:
df=df[['ga:dimension4','ga:avgSessionDuration','ga:pageviewsPerSession','ga:screenviewsPerSession']]#,'ga:avgTimeOnPage','ga:avgScreenviewDuration']]

In [475]:
df=df.sort_values(by='ga:dimension4', ascending=True)

In [476]:
df.shape

(963, 4)

In [128]:
#df['ga:avgScreenviewDuration'].describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4387 entries, 0 to 4386
Data columns (total 4 columns):
ga:dimension4               4387 non-null object
ga:avgSessionDuration       4387 non-null object
ga:avgTimeOnPage            4387 non-null object
ga:avgScreenviewDuration    4387 non-null object
dtypes: object(4)
memory usage: 171.4+ KB


In [82]:
assabo=pd.read_sas("D:/demandes_ponctuelles_py/RD/abo_numerique_AssAbo/assabo.sas7bdat", format='sas7bdat', encoding='latin-1')

In [83]:
extraction=pd.read_sas("D:/reporting_mensuel/datasets/wvn_extraction_201905.sas7bdat", format='sas7bdat', encoding='latin-1')

In [84]:
from datetime import datetime
from dateutil import relativedelta

extraction['Date_auj']=pd.Timestamp.today()

extraction['nbmois1']=extraction['Date_Fin']-extraction['Date_deb']
extraction['nbmois2']=extraction['Date_resiliation']-extraction['Date_deb']
extraction['nbmois3']=extraction['Date_auj']-extraction['Date_deb']

extraction['nbmois1_']=(extraction['nbmois1']/np.timedelta64(1,'M')).round(0)
extraction['nbmois2_']=(extraction['nbmois2']/np.timedelta64(1,'M')).round(0)
extraction['nbmois3_']=(extraction['nbmois3']/np.timedelta64(1,'M')).round(0)

In [161]:
#extraction['Date_deb'].dtype
#extraction['Date_auj'].dtype
#extraction.dtypes

In [85]:
extraction=extraction.drop(columns=['nbmois1', 'nbmois2', 'nbmois3'])

In [86]:
extraction.loc[(pd.notnull(extraction["Date_resiliation"])) & (extraction["Date_resiliation"] > extraction["Date_auj"]), "anciennete_mois"] = extraction['nbmois3_'] 
extraction.loc[(pd.isnull(extraction["Date_resiliation"])) & (pd.notnull(extraction["Date_Fin"])) & (extraction["Date_Fin"] > extraction["Date_auj"]), "anciennete_mois"] = extraction['nbmois3_'] 
extraction.loc[(pd.notnull(extraction["Date_Fin"])) & (pd.isnull(extraction["Date_resiliation"])) & (extraction["Date_Fin"] <= extraction["Date_auj"]), "anciennete_mois"] = extraction['nbmois1_'] 
extraction.loc[(pd.notnull(extraction["Date_resiliation"])) & (extraction["Date_resiliation"] < extraction["Date_auj"]), "anciennete_mois"] = extraction['nbmois2_'] 
extraction.loc[(pd.isnull(extraction["Date_resiliation"])) & (pd.isnull(extraction["Date_Fin"])), "anciennete_mois"] = extraction['nbmois3_'] 

#df[df.b.isnull()] # or notnull(), respectively
#cp_extraction[cp_extraction['depot'].isnull()

In [87]:
extraction=extraction.drop(columns=['nbmois1_', 'nbmois2_', 'nbmois3_','Date_auj'])

In [88]:
s=extraction.groupby(['Client_Dest']).Date_deb.transform('min')
extraction.loc[(extraction["Date_deb"] == s), "nouveau_abonne"] = 1
extraction.loc[(extraction["Date_deb"] != s), "nouveau_abonne"] = 0

In [89]:
extraction=extraction[extraction['code_offre'].str.contains("VNN0") & (extraction['date_origine']>='2018-05-01') & (extraction['date_origine']<'2019-01-01') & (extraction['Paiement'] != 'GR') & ((extraction['anciennete_mois'] == 3.0) | (extraction['anciennete_mois'] == 4.0) | (extraction['anciennete_mois'] == 5.0))]

In [138]:
extraction.shape

(747, 106)

In [90]:
decouverte=pd.read_csv("D:/demandes_ponctuelles_py/RD/offre_prospect.csv", header = 0, sep=";", encoding='latin1')

In [91]:
extraction_final=pd.merge(extraction, assabo, how='left', left_on=['Client_Dest','code_abo','increment'], right_on=['client_cust_id','vn_subscr_ast_id','vn_renewal_id'])

In [92]:
extraction_final_b=pd.merge(extraction_final, df, how='left', left_on=['uid_sso'], right_on=['ga:dimension4'])

In [93]:
extraction_final_c=pd.merge(extraction_final_b, decouverte, how='left', left_on=['uid_sso'], right_on=['SSO_ID'])

In [143]:
extraction_final_c

,Client_Dest,Role,Civ,nom_dest,prenom_dest,Nom,Adr_1,Adr_2,Adr_3,Adr_4,...,created,modified,vn_start_date,vn_delay,ga:dimension4,ga:avgSessionDuration,ga:avgTimeOnPage,ga:avgScreenviewDuration,TYPE,SSO_ID
0,00008387,9,MR,CARION,JEAN MARIE,"CARION,JEAN MARIE",NaN,NaN,7 BIS RUE DE MALEMPERCHE,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00066722,9,MME,WOESTELANDT,SYLVIE,"WOESTELANDT,SYLVIE",NaN,LIEU DIT LA BREARDE,1181 RUE DE STEENVOORDE,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,00072298,9,MR,PAPALSKI,EDOUARD,"PAPALSKI,EDOUARD",APT 21,RESIDENCE LES 3 PLACES,41 AVENUE SAINT EXUPERY,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,00090176,9,MME,BOURDON,CECILE,"BOURDON,CECILE",APT 35 RESIDENCE LES TERRASSES,NaN,10 RUE DES 528 DEPORTES JUIFS,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,00090358,9,MR,GERDUYN,PATRICK,"GERDUYN,PATRICK",NaN,NaN,1 A RUE DU FAISAN,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,00093297,9,MME,LOMBART,CATHERINE,"LOMBART,CATHERINE",NaN,NaN,27 RUE JULES WATTEEUW,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,00093297,9,MME,LOMBART,CATHERINE,"LOMBART,CATHERINE",NaN,NaN,27 RUE JULES WATTEEUW,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,00093297,9,MME,LOMBART,CATHERINE,"LOMBART,CATHERINE",NaN,NaN,27 RUE JULES WATTEEUW,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,00093297,9,MME,LOMBART,CATHERINE,"LOMBART,CATHERINE",NaN,NaN,27 RUE JULES WATTEEUW,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,00336915,2,NaN,MAIRIE,NaN,MAIRIE,NaN,LA GRANDE FERME,PLACE ALEXANDRE GRATTE,NaN,...,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
#extraction['date_origine'].describe
extraction_final_c.to_csv('ML.csv', sep=',', encoding='utf-8', index=False)